#### <b>MNIST ONNX Project</b>

* This code is from PyTorch's MNIST example (with only a few changes).
  * <b>Reference</b>: https://github.com/pytorch/examples/blob/master/mnist/main.py
* 본 코드는 MNIST 분류 모델을 학습한 뒤에, <b>오닉스(ONNX) 파일</b>로 내보내기까지 하는 코드입니다.
  * GPU를 사용하기 때문에, 런타임 유형을 GPU로 변경한 뒤에 실습을 진행합니다.

#### <b>Load Libraries</b>

In [1]:
import torch # PyTorch의 기본적인 라이브러리

import torch.nn as nn # Neural Network 그 자체
import torch.nn.functional as F # 다양한 함수(ReLU 등) 제공하는 라이브러리
import torch.optim as optim # 최적화(optimizer) 라이브러리

import torchvision # PyTorch를 이용해서 이미지/동영상을 처리하고자 할 때
from torchvision import datasets, transforms
# datasets: MNIST, CIFAR-10 등 다양한 데이터를 다운로드 및 불러와 사용
# transforms: 이미지 회전, 크기 변경 등 변형(transformation)

# 학습하는 과정에서 학습률(learning rate)를 점진적으로 줄여나가는 방식 사용
# StepLR은 특정한 epoch가 지날 때마다 단계적으로 감소시키는 방식
from torch.optim.lr_scheduler import StepLR

#### <b>Define Hyperparameters</b>



In [2]:
# 온점(.)으로 속성 값을 기입하도록 해주는 라이브러리
from types import SimpleNamespace

args = SimpleNamespace()

# 실질적인 하이퍼 파라미터 설정
args.batch_size = 512 # input batch size for training (default: 512)
args.test_batch_size = 1000 # input batch size for testing (default: 1000)
args.epochs = 10 # number of epochs to train (default: 10)
args.lr = 1.0 # learning rate (default: 1.0)
# 특정한 주기로 learning rate을 감소시킬 때, 몇 배수만큼씩 줄여나갈지
args.gamma = 0.7 # learning rate step gamma (default: 0.7)
# GPU를 사용할 것이기 때문에, 아래 값은 False로 기입
args.no_cuda = False # disables CUDA training
args.seed = 1 # random seed (default: 1)
args.log_interval = 10 # how many batches to wait before logging training status

use_cuda = not args.no_cuda and torch.cuda.is_available()

# visualize the argument parameters
args

namespace(batch_size=512,
          test_batch_size=1000,
          epochs=10,
          lr=1.0,
          gamma=0.7,
          no_cuda=False,
          seed=1,
          log_interval=10)

#### <b>Define Models</b>

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 입력 채널: 1, 출력 채널(커널의 개수): 32, 커널 크기: 3, stride: 1
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # 입력 채널: 32, 출력 채널(커널의 개수): 64, 커널 크기: 3, stride: 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # x: (batch_size, 28, 28, 1)
        x = self.conv1(x)
        x = F.relu(x)
        # x: (batch_size, 26, 26, 32)
        x = self.conv2(x)
        # x: (batch_size, 24, 24, 64)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        # x: (batch_size, 12, 12, 64)
        x = torch.flatten(x, 1)
        # x: (batch_size, 9216)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        # x: (batch_size, 128)
        x = self.fc2(x)
        # x: (batch_size, 10)
        return x

#### <b>Model Training Libraries</b>

In [13]:
def train(model, device, train_loader, optimizer, epoch):
    # 10개의 클래스를 가지므로, cross-entropy 손실(loss)
    criterion = nn.CrossEntropyLoss()
    model.train() # 모델을 학습 모드로 변경
    # 매 배치 단위로 데이터를 확인
    for batch_idx, (data, target) in enumerate(train_loader):
        # 입력 이미지와 정답 레이블을 GPU로 보내주기
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 모델의 가중치 기울기 초기화
        # 모델에 입력 이미지를 넣은 뒤에 손실(loss)을 계산
        output = model(data)
        loss = criterion(output, target)
        # 역전파(back-propagation)
        loss.backward()
        optimizer.step() # 모델의 가중치 업데이트
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    # 10개의 클래스를 가지므로, cross-entropy 손실(loss)
    criterion = nn.CrossEntropyLoss()
    model.eval() # 모델을 학습 모드로 변경
    test_loss = 0
    correct = 0
    # 모델을 학습하지 않고, 단순히 평가만 할 것이기 때문에 기울기 계산 X
    with torch.no_grad():
        # 매 배치 단위로 데이터를 확인
        for data, target in test_loader:
            # 입력 이미지와 정답 레이블을 GPU로 보내주기
            data, target = data.to(device), target.to(device)
            # 모델에 입력 이미지를 넣은 뒤에 정확도(accuracy) 계산
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#### <b>Define the Data Loader</b>

In [18]:
# 연구 목적의 상황과 다르게, 배포된 모델에 사람들이 입력 진행
train_transform = transforms.Compose([
    # add random transformations to the image
    # 다양한 각도와 크기에 대하여 강건할(robust) 필요가 있다.
    transforms.RandomAffine( # 랜덤하게 이미지를 변환
        degrees=10,
        translate=(0.0, 0.2),
        scale=(0.5, 1.2),
        shear=(-10, 10, -10, 10)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 테스트할 때는 입력 받은 이미지를 그대로 모델에 넣어주기
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('data', train=True, download=True, transform=train_transform)
test_dataset = datasets.MNIST('data', train=False, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

#### <b>Preview Dataset</b>

In [20]:
# 학습할 이미지가 어떻게 생겼는지 시각화
inputs_batch, labels_batch = next(iter(train_loader))
grid = torchvision.utils.make_grid(inputs_batch, nrow=40, pad_value=1)
torchvision.utils.save_image(grid, 'inputs_batch_preview.png')

#### <b>Run the Program</b>

In [21]:
# 실험할 때마다 결과가 달라지는 걸 원하지 않으므로
# 재현성(reproduciability)을 위해 시드(seed) 값 설정
torch.manual_seed(args.seed)

# GPU로 모델을 보내주어 학습할 것이기 때문에
device = torch.device("cuda" if use_cuda else "cpu")

# 실제로 학습할 모델을 초기화
model = Net().to(device)
# 학습할 때 사용할 최적화(optimizer) 도구
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

# 학습 진행
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

# <테스트 정확도가 낮을 수 있음>
# 이유: 테스트 데이터셋은 변형이 없는 올곧은 데이터로만 구성
# 우리는 현실 세계의 배포를 위해 데이터 증진을 강하게 적용
torch.save(model.state_dict(), "pytorch_model.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313355
Train Epoch: 1 [5120/60000 (8%)]	Loss: 1.917524
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.838268
Train Epoch: 1 [15360/60000 (25%)]	Loss: 1.450712
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.370872
Train Epoch: 1 [25600/60000 (42%)]	Loss: 0.892754
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.747283
Train Epoch: 1 [35840/60000 (59%)]	Loss: 0.686359
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.677472
Train Epoch: 1 [46080/60000 (76%)]	Loss: 0.657784
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.582088
Train Epoch: 1 [56320/60000 (93%)]	Loss: 0.541259

Test set: Average loss: 0.0003, Accuracy: 9131/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.672697
Train Epoch: 2 [5120/60000 (8%)]	Loss: 0.532947
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.563783
Train Epoch: 2 [15360/60000 (25%)]	Loss: 0.474682
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.432300
Train Epoch: 2 [25600/60000 (42%)]	Loss: 0.451109
Train Epoch: 2 [30720/60000 (51%)]	Loss: 0.482860
Tra

#### <b>Convert to ONNX Model</b>

In [22]:
# ONNX 파일로 내보내기 위해서 ONNX 라이브러리 설치
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 92.1 MB/s eta 0:00:00


In [23]:
# 오닉스(onnx) 배포 목적의 코드 작성
MEAN = 0.1307 # 원래 데이터 로더에 있던 코드
STANDARD_DEVIATION = 0.3081 # 원래 데이터 로더에 있던 코드


class InferenceNet(nn.Module):
  def __init__(self):
    super(InferenceNet, self).__init__()
    # 입력 채널: 1, 출력 채널(커널의 개수): 32, 커널 크기: 3, stride: 1
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    # 입력 채널: 32, 출력 채널(커널의 개수): 64, 커널 크기: 3, stride: 1
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    # 데이터 전처리 부분이 forward() 함수 앞쪽에 존재
    # <핵심> 데이터 전처리를 여기에 넣음
    # 웹 사이트의 JavaScript 입력 이미지 크기가 (280 X 280 X 4)
    # 채널이 4인 이유는? (RGBA) 이므로
    x = x.reshape(280, 280, 4)
    # 흑백 이미지로 만드는 코드
    x = torch.narrow(x, dim=2, start=3, length=1)
    # PyTorch Vision의 입력은 항상 다음과 같다.
    # (batch_size, channel_size, width, height)
    x = x.reshape(1, 1, 280, 280)
    # 학습한 모델은 (28 X 28)의 크기를 받기 때문에 조절
    x = F.avg_pool2d(x, 10, stride=10)
    x = x / 255 # PyTorch는 [0, 1]의 값만 받으므로
    # 정규화(normalization)
    x = (x - MEAN) / STANDARD_DEVIATION

    # x: (batch_size, 28, 28, 1)
    x = self.conv1(x)
    x = F.relu(x)
    # x: (batch_size, 26, 26, 32)
    x = self.conv2(x)
    # x: (batch_size, 24, 24, 64)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    # x: (batch_size, 12, 12, 64)
    x = torch.flatten(x, 1)
    # x: (batch_size, 9216)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    # x: (batch_size, 128)
    x = self.fc2(x)
    # x: (batch_size, 10)
    # 배포할 때는 확률(probability)을 뱉는 것이 이상적
    # 소프트맥스(softmax)를 거친 결과를 반환
    output = F.softmax(x, dim=1)
    return output

In [29]:
# 추론(inference) 목적의 네트워크를 초기화한다.
pytorch_model = InferenceNet()

# 기본적으로 model의 가중치(__init__)의 구조가 일치한다면, 불러올 수 있다.
# forward() 함수의 형식이 달라도 정상적으로 불러오는 것이 가능하다.
pytorch_model.load_state_dict(torch.load('pytorch_model.pt'))
pytorch_model.eval()

dummy_input = torch.zeros(280 * 280 * 4)

pytorch_model(dummy_input)

tensor([[1.1405e-04, 9.9968e-01, 8.0074e-07, 1.3354e-06, 2.1442e-05, 9.2801e-05,
         3.8494e-05, 5.0200e-05, 3.1511e-07, 5.0974e-07]],
       grad_fn=<SoftmaxBackward0>)

In [30]:
# 실제 ONNX 파일로 내보내기
torch.onnx.export(pytorch_model, dummy_input, 'onnx_model.onnx', verbose=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

